In [221]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def genMockDataFrame(
    days,
    start_price,
    colName,
    startDate,
    seed=None,
):
    periods = days * 24
    np.random.seed(seed)

    P = np.around(
        start_price + np.random.normal(scale=0.0018, size=periods).cumsum(), 4
    )
    P = [round(i, 4) for i in P]

    fxDF = pd.DataFrame(
        {
            "ticker": [colName] * periods,
            "price": P,
        },
        index=pd.date_range(startDate, periods=periods, freq="H"),
    )

    return fxDF.price.resample("D").ohlc()


df = genMockDataFrame(100, 1.1904, "BTCUSDT", "19/3/2020", seed=None)

fig = go.Figure(
    data=go.Candlestick(
        x=df.index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
    )
)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [241]:
def genMockDataFrame(
    days,
    start_price,
    colName,
    startDate,
    seed=None,
):
    periods = days * 24
    np.random.seed(seed)

    P = np.around(
        start_price + np.random.normal(scale=0.0018, size=periods).cumsum(), 4
    )
    V = np.abs(np.random.normal(loc=1000000, scale=500000, size=periods))
    P = [round(i, 4) for i in P]

    fxDF = pd.DataFrame(
        {
            "ticker": [colName] * periods,
            "price": P,
            "volume": V,
        },
        index=pd.date_range(
            startDate,
            periods=periods,
            freq="H",
        ),
    )

    # Group the data by day and calculate OHLCV
    ohlc_df = fxDF.groupby(pd.Grouper(freq="D")).agg(
        {
            "price": ["first", "max", "min", "last"],
            "volume": "sum",
        }
    )
    ohlc_df.columns = [
        "open",
        "high",
        "low",
        "close",
        "volume",
    ]

    return ohlc_df


df = genMockDataFrame(
    100,
    1.1904,
    "BTCUSDT",
    "19/3/2023",
)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    row_heights=[0.75, 0.25],
)

fig.add_trace(
    go.Ohlc(
        x=df.index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name="BTCUSDT",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(x=df.index, y=df["volume"], name="Volume"),
    row=2,
    col=1,
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()